# Introduction to Pygama Math and Fitting

The goal of this notebook is to illustrate the conventions of the functions stored in `pygama.math` and how they can be used to fit data. We will also go over how to write new math distributions. 

### Set up the Python environment

In [ ]:
import pygama.math as math

import matplotlib.pyplot as plt
import numpy as np
import os

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 12

## Statistical Distributions 

All statistical distributions are stored in `pygama.math.functions` and can be imported through the module `pygama.math.distributions`. Let's import a plain-old Gaussian and look at the methods it has associated with it! And then, at the end of this section, we can look at the conventions and how to write a new distribution. 

Let's also fix the shape parameters for the Gaussian.

For comparison's sake, let's also import `scipy`'s version of the Gaussian. 

In [ ]:
from pygama.math.distributions import gaussian
from scipy.stats import norm

mu = 2.5
sigma = 0.7
x = np.linspace(-10, 10, 100)

object_methods = [
    method_name
    for method_name in dir(gaussian)
    if callable(getattr(gaussian, method_name))
]
print(object_methods)

Geez, there are a lot of methods. Let's hone in on a couple: 
### .pdf and .cdf 

These are just your bread and butter probability density and cumulative density functions

In [ ]:
gauss_pdf = gaussian.pdf(x, mu, sigma)

plt.plot(x, gauss_pdf, label="Pygama Gaussian PDF", c="k")
plt.plot(x, norm.pdf(x, mu, sigma), label="Scipy Gaussian PDF", ls=":", alpha=1, c="r")
plt.title("PDF of a Gaussian Distribution")
plt.legend()
plt.show()

In [ ]:
gauss_cdf = gaussian.cdf(x, mu, sigma)

plt.plot(x, gauss_cdf, label="Gaussian CDF", c="k")
plt.plot(x, norm.cdf(x, mu, sigma), label="Scipy Gaussian CDF", ls=":", alpha=1, c="r")
plt.title("CDF of a Gaussian Distribution")
plt.legend()
plt.show()

Ok, so that's fine and dandy, what's so special about a class that reproduces `scipy` results? 

Well, every function in `pygama.math.functions` subclasses `scipy`'s `rv_continuous` class: this allows access to methods that `scipy` automagically computes. Let's look at a few 

### What `rv_continuous` subclassing gets us: `.rvs`, `.logpdf`, `.sf`... 

In [ ]:
gauss_log_pdf = gaussian.logpdf(x, mu, sigma)

plt.plot(x, gauss_log_pdf, label="Gaussian Log PDF", c="k")
plt.plot(
    x, norm.logpdf(x, mu, sigma), label="Scipy Gaussian Log PDF", ls=":", alpha=1, c="r"
)
plt.title("Log PDF of a Gaussian Distribution")
plt.legend()
plt.show()

In [ ]:
gauss_sf = gaussian.sf(x, mu, sigma)

plt.plot(x, gauss_sf, label="Gaussian SF", c="k")
plt.plot(x, norm.sf(x, mu, sigma), label="Scipy Gaussian SF", ls=":", alpha=1, c="r")
plt.title("Survival Fraction of a Gaussian Distribution")
plt.legend()
plt.show()

And now for something really useful, random sampling: 

In [ ]:
random_sample = gaussian.rvs(mu, sigma, size=1000, random_state=1)

hist, bins = np.histogram(random_sample, bins=100, range=(-10, 10))
plt.step(
    bins[1:], hist / np.sum(hist) * np.sum(gauss_pdf), label="Randomly Sampled Data"
)
plt.plot(x, gauss_pdf, label="Gaussian PDF")
plt.title("Comparison of Randomly Sampled Data with Underlying PDF")
plt.legend()
plt.show()

## We get access to `scipy`'s `rv_continuous` methods for free

If we take a look at the actual code for the definition of `pygama.math.functions.gaussian` we see that there is no direct implementation of methods for `.rvs` or `.logpdf`! We get access to all these methods for free. All we have to do is subclass `rv_continuous` and write definitions for `_pdf` and `_cdf` 

Now, the drawback: the functions `_pdf` and `_cdf` that `rv_continuous` uses are slow. Let's see this by comparing to the function that actually computes the pdf: 
### But the `.pdf` and `.cdf` methods are slow! 

In [ ]:
%%timeit -r 4 -n 10000
gauss_pdf = gaussian.pdf(x, mu, sigma)

In [ ]:
def gaussian_pdf(x, mu, sigma):
    if sigma == 0:
        invs = np.inf
    else:
        invs = 1.0 / sigma
    z = (x - mu) * invs
    invnorm = invs / np.sqrt(2 * np.pi)
    return np.exp(-0.5 * z**2) * invnorm

In [ ]:
%%timeit -r 4 -n 10000
gauss_pdf = gaussian_pdf(x, mu, sigma)

## Fast methods: `.get_pdf` and `.get_cdf`

Because the overloaded `rv_continuous` implementations of `pdf` and `cdf` are very slow, we have implemented fast versions of these two methods, named `get_pdf` and `get_cdf`. 

Each of these fast methods call a `numba.njit`-wrapped function --- a just-in-time compiled function --- that runs super fast. 

We keep the `.pdf` and `.cdf` because they give us access to `scipy` methods we might want to use, but we write these faster methods so that we can fit distributions quickly. 

In [ ]:
gauss_get_pdf = gaussian.get_pdf(x, mu, sigma)

plt.plot(x, gauss_get_pdf, label="Pygama Gaussian Get_PDF", c="k")
plt.plot(x, norm.pdf(x, mu, sigma), label="Scipy Gaussian PDF", ls=":", alpha=1, c="r")
plt.title("PDF of a Gaussian Distribution")
plt.legend()
plt.show()

Ok, that's good that it reproduces the scipy results numerically. But how fast does it run?

In [ ]:
%%timeit -r 4 -n 10000
gauss_pdf = gaussian.get_pdf(x, mu, sigma)

See, we're even faster than the function we defined above! That's the power of just-in-time compiled code

## The remaining methods are helpful for binned and unbinned fitting: `norm_pdf`, `norm_cdf`, `pdf_ext` and `cdf_ext`

For `pygama` our default way of fitting distributions is to use the `Iminuit` package. `Iminuit` has several different ways to fit binned and unbinned data by performing either extended or unextended fits. These fitting methods require functions with different properties. The way `pygama` functions interact with `Iminuit` is as follows: 

1. `norm_pdf` is used for unbinned fits, which require a pdf that is normalized to 1 on the fitting range 
2. `norm_cdf` is used for binned fits, which require a cdf that derived from a pdf that is normalized to 1 on the fitting range 
3. `pdf_ext` is used for extended unbinned fits, and the function is required to return a tuple of the support-normalized pdf integrated over the data window, and the scaled support-normalized pdf.
4. `cdf_ext` is used for extended binned fits, and just returns the scaled support-derived cdf

Let's illustrate some fitting in action, and why some of these weird definitions are required 
### Import Iminuit and create data to fit 

In [ ]:
from iminuit import cost, Minuit

xr = (-4, 4)  # xrange
mu = 0.5
sigma = 3

rng = np.random.default_rng(1)

xdata = rng.normal(mu, sigma, size=1000)
xdata = xdata[(xr[0] < xdata) & (xdata < xr[1])]

n, xe = np.histogram(xdata, bins=50, range=xr)
cx = 0.5 * (xe[1:] + xe[:-1])
dx = np.diff(xe)

plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.plot(xdata, np.zeros_like(xdata), "|", alpha=0.1);

### Try an unbinned fit using the correct `norm_pdf` and the incorrect `get_pdf`

We first note that the method `norm_pdf` has arguments `x_lower, x_upper, mu, sigma` where `x_lower, x_upper` are the bounds of the fitting range to ensure that the pdf is normalized to unity on. So, we therefore fix these fit parameters in Iminuit. 

We also show a fit with `get_pdf` to show that the fit returns the incorrect results! We keep the `get_pdf` methods in `pygama` because they are useful if we ever need to numerically compute something quickly, or if we do a least squares fit. 

In [ ]:
c = cost.UnbinnedNLL(xdata, gaussian.norm_pdf)

m_norm = Minuit(c, x_lower=xr[0], x_upper=xr[1], mu=0.4, sigma=0.2)
m_norm.fixed["x_lower", "x_upper"] = True
m_norm.limits["mu", "sigma"] = (0, None)
m_norm.migrad()

In [ ]:
c = cost.UnbinnedNLL(xdata, gaussian.get_pdf)

m = Minuit(c, mu=0.4, sigma=0.2)
m.limits["mu", "sigma"] = (0, None)
m.migrad()

In [ ]:
plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(
    xm, gaussian.norm_pdf(xm, *m_norm.values) * len(xdata) * dx[0], label="norm_pdf fit"
)
plt.plot(xm, gaussian.get_pdf(xm, *m.values) * len(xdata) * dx[0], label="get_pdf fit")
plt.plot(
    xm, gaussian.get_pdf(xm, mu, sigma) * len(xdata) * dx[0], label="True Distribution"
)
plt.legend()
plt.show()

We see that `norm_pdf` does a much better job of fitting the actual underlying distribution and reproducing the correct $\mu,\sigma$ parameters

### try an extended unbinned fit 
The parameters for the `pdf_ext` are `area, x_lo, x_hi, mu, sigma`

In [ ]:
c = cost.ExtendedUnbinnedNLL(xdata, gaussian.pdf_ext)

m = Minuit(c, area=500, x_lo=xr[0], x_hi=xr[1], mu=0.1, sigma=0.9)
m.fixed["x_lo", "x_hi"] = True
m.limits["area", "mu", "sigma"] = (0, None)
m.migrad()

In [ ]:
plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(xm, gaussian.pdf_ext(xm, *m.values)[1] * dx[0], label="fit")
plt.plot(
    xm, gaussian.pdf_ext(xm, 1000, xr[0], xr[1], mu, sigma)[1] * dx[0], label="actual"
)
plt.legend()
plt.show()

The `pdf_ext` function correctly fits the area of the curve! 

### Binned fits using `norm_cdf` and the incorrect `get_cdf`

In [ ]:
c = cost.BinnedNLL(n, xe, gaussian.norm_cdf)

m_norm = Minuit(c, x_lower=xr[0], x_upper=xr[1], mu=0.4, sigma=0.2)
m_norm.fixed["x_lower", "x_upper"] = True
m_norm.limits["mu", "sigma"] = (0.1, None)
m_norm.migrad()

In [ ]:
c = cost.BinnedNLL(n, xe, gaussian.get_cdf)

m = Minuit(c, mu=0.4, sigma=0.2)
m.limits["mu", "sigma"] = (0.1, None)
m.migrad()

In [ ]:
plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.stairs(
    np.diff(gaussian.norm_cdf(xe, *m_norm.values)) * len(xdata),
    xe,
    label="norm_cdf fit",
)
plt.stairs(
    np.diff(gaussian.get_cdf(xe, *m.values)) * len(xdata), xe, label="get_cdf fit"
)
plt.stairs(
    np.diff(gaussian.get_cdf(xe, mu, sigma)) * len(xdata),
    xe,
    label="underlying distribution",
)
plt.legend()
plt.show()

### Finally, extended binned fits with `cdf_ext`

In [ ]:
c = cost.ExtendedBinnedNLL(n, xe, gaussian.cdf_ext)

m = Minuit(c, area=500, mu=0.1, sigma=0.9)
m.limits["area", "mu", "sigma"] = (0, None)
m.migrad()

In [ ]:
plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.stairs(np.diff(gaussian.cdf_ext(xe, *m.values)), xe, label="fit")
plt.stairs(np.diff(gaussian.cdf_ext(xe, 1000, mu, sigma)), xe, label="underlying")
plt.legend();

# Writing our own `pygama.math` distribution: conventions and requirements 

Let's write a Cauchy distribution. Recall that a Cauchy distribution has support over the real line, and has a pdf like 

$pdf(x, \mu,\sigma) = \frac{1}{\pi\sigma\left[1+\left(\frac{x-\mu}{\sigma}\right)^2\right]}$

and a cdf:

$cdf(x, \mu,\sigma) = \frac{1}{\pi}\arctan\left(\frac{x-\mu}{\sigma}\right)+\frac{1}{2}$

## What we need to define a distribution class
We need four functions that our class methods will call and their arguments, and they all should be numbafied 

1. A PDF, normalized to the support. Args: x, mu, sigma
2. A CDF, derived from the support normalized PDF. Args: x, mu, sigma
3. A scaled PDF. Args: x, area, mu, sigma
4. A scaled CDF. Args: x, area, mu, sigma

The convention is to name these functions `nb_distribution_pdf` and `nb_distribution_scaled_cdf` for example.

In [ ]:
import numba as nb

# here we set some parameters to ensure that numba will compute things as quickly as possible
kwd_parallel = {"parallel": True, "fastmath": True}
kwd = {"parallel": False, "fastmath": True}

# define the pdf
@nb.njit(**kwd_parallel)
def nb_cauchy_pdf(x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
    r"""
    Parameters
    ----------
    x
        The input data
    lamb
        The rate
    mu
        The amount to shift the distribution
    sigma
        The amount to scale the distribution
    """

    y = np.empty_like(x, dtype=np.float64)
    # we want to do a loop because it is faster with parallelization
    for i in nb.prange(x.shape[0]):
        y[i] = (x[i] - mu) / sigma
        y[i] = 1 / (np.pi * sigma * (1 + y[i] ** 2))
    return y


# define the cdf
@nb.njit(**kwd_parallel)
def nb_cauchy_cdf(x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
    r"""
    Parameters
    ----------
    x
        The input data
    lamb
        The rate
    mu
        The amount to shift the distribution
    sigma
        The amount to scale the distribution
    """
    y = np.empty_like(x, dtype=np.float64)
    for i in nb.prange(x.shape[0]):
        y[i] = (x[i] - mu) / sigma
        y[i] = (1 / np.pi) * np.arctan(y[i]) + 0.5
    return y


# define the scaled pdf, can't use parallelization here because there is no outer for-loop
@nb.njit(**kwd)
def nb_cauchy_scaled_pdf(
    x: np.ndarray, area: float, mu: float, sigma: float
) -> np.ndarray:
    r"""
    Parameters
    ----------
    x
        The input data
    area
        The prefactor to scale the pdf by
    lamb
        The rate
    mu
        The amount to shift the distribution
    sigma
        The amount to scale the distribution
    """
    return area * nb_cauchy_pdf(x, mu, sigma)


# define the scaled cdf, can't use parallelization here because there is no outer for-loop
@nb.njit(**kwd)
def nb_cauchy_scaled_cdf(
    x: np.ndarray, area: float, mu: float, sigma: float
) -> np.ndarray:
    r"""
    Parameters
    ----------
    x
        The input data
    area
        The prefactor to scale the pdf by
    lamb
        The rate
    mu
        The amount to shift the distribution
    sigma
        The amount to scale the distribution
    """
    return area * nb_cauchy_cdf(x, mu, sigma)

### Now we create a class using these four functions 

Each `pygama` distribution class subclasses `pygama_continuous`, which itself is a subclass of `rv_continuous`. 

#### An aside on the ordering of parameters: 
In the following the term "shape parameter" refers to a specific variable used to define a distribution that is not an overall scaling (which I call $\mu$) or shifting (which I call $\sigma$). An example of shape parameters would be the $\beta, m$ parameters in a crystal ball function. The location and scale ($\mu, \sigma$) parameters have a more specific definition --- akin to what `scipy` does. If a $\mu, \sigma$ value is present, then the entire distribution is shifted and scaled; this can be achieved by transforming $y = \frac{x-\mu}{\sigma}$ and computing $pdf(y)/\sigma$ ( where the factor of $1/\sigma$ comes from the Jacobian). 

The ordering of parameters is as follows, using whichever are needed for a function: 
`x, area, x_lower, x_upper, shapes, ..., mu, sigma`

Where `x_lower` and `x_upper` are the lower and upper bounds to evaluate a function on.

#### Now, back to require methods

The class name has the format `distribution_name_gen` and it must include the following methods with their arguments:

1. `_pdf(self, x, shapes)`:

    This is an overloading of the `scipy` method for the pdf. Because `scipy` first computes $\frac{x-\mu}{\sigma}$ and $pdf/\sigma$, we need to call `nb_distribution_pdf` with $\mu = 0$ and $\sigma = 1$. If shape parameters are present, `nb_distribution_pdf` must also be called with `shape_param[0]` because `scipy` passes an array of shape parameters to the function calls. Finally, `x.flags.writeable = True` must be passed so that `numba` can operate as expected on arrays. 
    
  
2. `_cdf(self, x, shapes)`:

    This is an overloading of the `scipy` method for the cdf. The same format applies as for the pdf. 
       
       
3. `get_pdf(x, shapes, mu, sigma)`: 

    This is a direct call to the fast `nb_distribution_pdf`
  
  
4. `get_pdf(x, shapes, mu, sigma)`: 

    This is a direct call to the fast `nb_distribution_cdf`
    
    
5. `norm_pdf(x, x_lower, x_upper, shapes, mu, sigma)`:

    In the case that the support of the distribution is the entire real line, this calls a `pygama_continuous` super method that normalizes the pdf to unity on the range $[x_{lower},x_{upper}]$ by computing $\frac{pdf(x)}{cdf(x_{upper})- cdf(x_{lower})}$. If the distribution is defined on a limited domain, this function needs to be directly overloaded -- see `pygama.math.functions.step` for an example. 
    
    
6. `norm_cdf(x, x_lower, x_upper, shapes, mu, sigma)`:

    In the case that the support of the distribution is the entire real line, this calls a `pygama_continuous` super method that derives the cdf from a pdf that is normalized to unity on the range $[x_{lower},x_{upper}]$ by computing $\frac{cdf(x)}{cdf(x_{upper})- cdf(x_{lower})}$. If the distribution is defined on a limited domain, this function needs to be directly overloaded -- see `pygama.math.functions.step` for an example.
    
    
7. `pdf_ext(x, area, x_lower, x_upper, shapes, mu, sigma)`: 

    A function that returns both the integral of the support-normalized pdf on the interval $[x_{lower},x_{upper}]$, as well as the support-normalized pdf on that range as well. The fastest way to compute these is to return `np.diff(nb_distribution_scaled_cdf(np.array([x_lower, x_upper]), area, shapes, mu, sigma))` and `nb_distribution_scaled_pdf(x, area, shapes, mu, sigma)`
  
  
8. `cdf_ext(x, area, shapes, mu, sigma)`:

    A function that returns just the scaled cdf, `nb_distribution_scaled_cdf(x, area, shapes, mu, sigma)`
    
    
9. `req_args(self)`: 

    A function that returns a tuple with the strings of the names of the required shape parameters and mu and sigma


In [ ]:
from pygama.math.functions.pygama_continuous import pygama_continuous


class cauchy_gen(pygama_continuous):
    def _pdf(self, x: np.ndarray) -> np.ndarray:
        x.flags.writeable = True
        return nb_cauchy_pdf(x, 0, 1)

    def _cdf(self, x: np.ndarray) -> np.ndarray:
        x.flags.writeable = True
        return nb_cauchy_cdf(x, 0, 1)

    def get_pdf(self, x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
        return nb_cauchy_pdf(x, mu, sigma)

    def get_cdf(self, x: np.ndarray, mu: float, sigma: float) -> np.ndarray:
        return nb_cauchy_cdf(x, mu, sigma)

    # needed so that we can hack iminuit's introspection to function parameter names.
    def norm_pdf(
        self, x: np.ndarray, x_lower: float, x_upper: float, mu: float, sigma: float
    ) -> np.ndarray:
        return self._norm_pdf(x, x_lower, x_upper, mu, sigma)

    def norm_cdf(
        self, x: np.ndarray, x_lower: float, x_upper: float, mu: float, sigma: float
    ) -> np.ndarray:
        return self._norm_cdf(x, x_lower, x_upper, mu, sigma)

    def pdf_ext(
        self,
        x: np.ndarray,
        area: float,
        x_lower: float,
        x_upper: float,
        mu: float,
        sigma: float,
    ) -> np.ndarray:
        return np.diff(
            nb_cauchy_scaled_cdf(np.array([x_lower, x_upper]), area, mu, sigma)
        ), nb_cauchy_scaled_pdf(x, area, mu, sigma)

    def cdf_ext(
        self, x: np.ndarray, area: float, mu: float, sigma: float
    ) -> np.ndarray:
        return nb_cauchy_scaled_cdf(x, area, mu, sigma)

    def required_args(self) -> tuple[str, str]:
        return "mu", "sigma"


cauchy = cauchy_gen(name="cauchy")

In [ ]:
from scipy.stats import cauchy as scipy_cauchy

cauchy_pdf = cauchy.pdf(x, mu, sigma)
cauchy_get_pdf = cauchy.get_pdf(x, mu, sigma)

cauchy_cdf = cauchy.cdf(x, mu, sigma)
cauchy_get_cdf = cauchy.get_cdf(x, mu, sigma)

plt.plot(x, cauchy_pdf, label="Pygama Cauchy PDF", c="k")
plt.plot(x, cauchy_get_pdf, label="Pygama Cauchy get_pdf", c="b", ls="--")
plt.plot(
    x, scipy_cauchy.pdf(x, mu, sigma), label="Scipy Cauchy PDF", ls=":", alpha=1, c="r"
)
plt.title("PDF of a Cauchy Distribution")
plt.legend()
plt.show()


plt.plot(x, cauchy_cdf, label="Pygama Cauchy CDF", c="k")
plt.plot(x, cauchy_get_cdf, label="Pygama Cauchy get_cdf", c="b", ls="--")
plt.plot(
    x, scipy_cauchy.cdf(x, mu, sigma), label="Scipy Cauchy CDF", ls=":", alpha=1, c="r"
)
plt.title("CDF of a Cauchy Distribution")
plt.legend()
plt.show()

### Perform some fits using the new Cauchy distribution to show the other methods' validity 

In [ ]:
xr = (-4, 4)  # xrange
mu = 0.5
sigma = 0.7

xdata = scipy_cauchy.rvs(mu, sigma, size=1000, random_state=42)
xdata = xdata[(xr[0] < xdata) & (xdata < xr[1])]

n, xe = np.histogram(xdata, bins=50, range=xr)
cx = 0.5 * (xe[1:] + xe[:-1])
dx = np.diff(xe)

plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.plot(xdata, np.zeros_like(xdata), "|", alpha=0.1);

### unbinned fit

In [ ]:
c = cost.UnbinnedNLL(xdata, cauchy.norm_pdf)

m_norm = Minuit(c, x_lower=xr[0], x_upper=xr[1], mu=0.4, sigma=0.2)
m_norm.fixed["x_lower", "x_upper"] = True
m_norm.limits["mu", "sigma"] = (0, None)
m_norm.migrad()

### extended unbinned fit

In [ ]:
c = cost.ExtendedUnbinnedNLL(xdata, cauchy.pdf_ext)

m = Minuit(c, area=500, x_lower=xr[0], x_upper=xr[1], mu=0.1, sigma=0.9)
m.fixed["x_lower", "x_upper"] = True
m.limits["area", "mu", "sigma"] = (0.01, None)
m.migrad()

### Binned fit

In [ ]:
c = cost.BinnedNLL(n, xe, cauchy.norm_cdf)

m_norm = Minuit(c, x_lower=xr[0], x_upper=xr[1], mu=0.4, sigma=0.2)
m_norm.fixed["x_lower", "x_upper"] = True
m_norm.limits["mu", "sigma"] = (0.1, None)
m_norm.migrad()

### Extended binned fit 

In [ ]:
c = cost.ExtendedBinnedNLL(n, xe, cauchy.cdf_ext)

m = Minuit(c, area=500, mu=0.1, sigma=0.9)
m.limits["area", "mu", "sigma"] = (0, None)
m.migrad()

# Adding distributions with `sum_dists`

In the business of fitting data, adding two distributions together is our bread-and-butter. This part of the notebook will show you how to create your own distribution that subclasses `pygama.math.sum_dists`. There are a couple of different use cases for adding distributions together, and we will look at each in turn, but here is a summary:

1. Adding different fractions of distributions together, and fitting out the relative amount of distributions
2. Adding distributions with different areas (present in equal fractions) and fitting the areas 
3. Adding distributions with different areas that have different fractions, and fitting out the areas and fractions 

### Let's first create the synthetic data that we will fit 

In [ ]:
xr = (-4, 4)  # xrange

rng = np.random.default_rng(1)
mu = 0.5
sigma = 1.3

mu2 = 1
sigma2 = 0.6


xdata = norm.rvs(mu, sigma, size=1000, random_state=42)
ydata = scipy_cauchy.rvs(mu2, sigma2, size=len(xdata), random_state=42)
xmix = np.append(xdata, ydata)
xmix = xmix[(xr[0] < xmix) & (xmix < xr[1])]

n, xe = np.histogram(xmix, bins=50, range=xr)
cx = 0.5 * (xe[1:] + xe[:-1])
dx = np.diff(xe)

plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.plot(xmix, np.zeros_like(xmix), "|", alpha=0.1);

# A little about `sum_dists`
The first important thing to note about this class is that all method class are of the form `method(x, parameter_array)`. We want to use a single array as an argument because we need to accommodate an arbitrarily large number of parameters. In addition, a single array is an acceptable input for `Iminuit` fits. 

So, the first thing a user does in creating a new class is define the order of elements in a `parameter_array`. Now, `sum_dists` works by grabbing elements at different indices in this `parameter_array` according to rules that the user provides in the `__init__`. Here's how to write a generic class: 

1. Create a `parameter_index_array` that holds the indices of what will eventually come in the `parameter_array`. If the user will eventually pass `parameters = [frac1, mu, sigma]` then we just take `parameter_index_array=[frac1, mu, sigma]=range(3)`

2. `sum_dists` takes an alternating pattern of distributions and distribution-specific parameter_index_arrays. Each par array can contain `[shape, mu, sigma, area, frac]` with area and frac being optional (depending on the flag sent to the constructor) but *must be placed in that order*. 

3. We pass one of the 4 flag options, to be described below. 

Let's trace through how the code works for a simple example. Suppose we want to create the following distribution 

$new\_pdf(x, [\tau, \mu, \sigma, frac_1, frac_2]) = frac_1\cdot dist_1(x, \tau, \mu, \sigma) + frac_2\cdot dist_2(x, \mu, \sigma)$

The first thing we would do is create our `parameter_index_array` 

`[tau, mu, sigma, frac_1, frac_2] = range(5)`

Then, we would create our alternating pattern of distributions and parameter index arrays:

`args = [dist1, [tau, mu, sigma, frac_1], dist2, [mu, sigma, frac2]]`

Finally, we would intitalize (with the `fracs` flag in this case, more on that later) 

`super().__init__(*args, frac_flag = "fracs")`

## So... What is `sum_dists` actually doing? 
Under the hood, `sum_dists` is applying a set of rules so that the following *is always* computed, regardless of the flag sent to the constructor. 

`total_area*area1*frac1*dist1(x, shape, mu, sigma) + ... total_area*area_n*frac_n*dist_n(x, shape_1, mu_n, sigma_n)`

It does this by first reading through all of the parameter index arrays for each distribution and separating the indices for the shape parameters, areas, and fracs into separate index arrays. Then, at the time of method call, these separate parameter index arrays are used to grab the actual values for the shape parameters, fractions, and areas. 
There's also some work done to determine which of `area` and `frac` are present. That's the purpose of the `frac_flag`. Let's take some time and learn a little more about what each flag does. 
 


## The `areas` flag in the `sum_dists` constructor
Let's say we are interested in knowing the amount of counts present in a signal and a background in our total spectrum, i.e. we want to create and fit a function that looks like

$pdf= A\cdot gauss\_pdf + B\cdot cauchy\_pdf$

Because we are interested in fitting the areas, we send the `areas` keyword to `frac_flag`. This causes `sum_dists` to assume that *the last element in each parameter index array in the constructor is an area for that specific distribution*. By default, all the `fracs` are set to 1, as well as the `total_area`.


In [ ]:
from pygama.math.functions.sum_dists import sum_dists


class cauchy_on_gauss_gen(sum_dists):
    def __init__(self):
        # we first create an array contains the indices of the parameter array
        # that will eventually be input to function calls
        (area1, mu, sigma, area2, mu2, sigma2) = range(6)

        # we now create an array containing the distributions and their shape parameters
        # The last item in a parameter array is the fraction or area component
        # The last item in the last parameter array corresponds to the total area, if present
        # but because we are fitting individual areas, the total area is meaningless
        args = [gaussian, [mu, sigma, area1], cauchy, [mu2, sigma2, area2]]
        # we initialize with the frac_flag = "areas" to let the constructor know we are sending frac parameters only
        super().__init__(*args, frac_flag="areas")

    def get_req_args(self) -> tuple[str, str, str]:
        r"""
        Return the required arguments for this instance
        """
        return "area1", "mu", "sigma", "area2", "mu2", "sigma2"


cauchy_on_gauss = cauchy_on_gauss_gen()

### That was super quick, let's see how it does with fitting data

Because we are interested in fitting out areas, we need to use the extended forms of our fits 
And for `pdf_ext` methods we have two options: we either pass `x_lo, x_hi` at the start of our parameter array, or we don't pass them and the function automatically takes the `np.amin(x), np.amax(x)` as those values 

## something to note: all parameter arrays passed to `sum_dist` methods *must* be `numpy` arrays; for example, see that we must turn `m.values` into `np.array(m.values)` in order for it to work

In [ ]:
c = cost.ExtendedUnbinnedNLL(xmix, cauchy_on_gauss.pdf_ext)

m = Minuit(c, (xr[0], xr[1], 100, 0.1, 0.3, 100, 1, 2))
m.fixed[0, 1] = True  # fix the x_lo, x_hi
m.limits[2, 5] = (0, None)
m.limits[3, 4, 6, 7] = (0, 2)
print(m.migrad())

plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(xm, cauchy_on_gauss.pdf_ext(xm, np.array(m.values))[1] * dx[0], label="fit")
plt.plot(
    xm,
    cauchy_on_gauss.pdf_ext(xm, np.array([1000, mu, sigma, 1000, mu2, sigma2]))[1]
    * dx[0],
    label="actual",
)
plt.legend();

Well, that's pretty good! Let's see how the extended unbinned fit fairs

In [ ]:
c = cost.ExtendedBinnedNLL(n, xe, cauchy_on_gauss.cdf_ext)
m = Minuit(c, (100, 0.1, 0.3, 100, 1, 2))
m.limits[0, 3] = (0, None)
m.limits[1, 2, 4, 5] = (0, 2)
print(m.migrad())

plt.errorbar(cx, n, n**0.5, fmt="ok")
plt.stairs(np.diff(cauchy_on_gauss.cdf_ext(xe, np.array(m.values))), xe, label="fit")
plt.stairs(
    np.diff(
        cauchy_on_gauss.cdf_ext(xe, np.array([1000, mu, sigma, 1000, mu2, sigma2]))
    ),
    xe,
    label="actual",
)
plt.legend();

## The `fracs` flag in the `sum_dists` constructor
To get a feel for how `sum_dists` works, let's create a function that creates the following distribution:

$pdf = f_1\cdot gauss\_{pdf}+(1-f_1)\cdot cauchy\_{pdf}$

Well, it might seem crazy, but we can actually link together parameters before they are fit! That's the beauty of the parameter index array and the `link_pars` function that we can overload.

The `fracs` keyword causes `sum_dists` to assume the following about the parameter index arrays in the `__init__`: each parameter index array looks like `[shapes, mu, sigma, frac]` with the exception of the last parameter index array which has the form `[shapes, mu, sigma, frac, total_area]`. 

Well, what if we don't care about `total_area`? Or, what if we want to link parameters, like the `f_1` value in the above example? That's where `link_pars` comes in. Nominally, `link_pars` serves to extract the actual values of the shapes, fracs, and areas from the passed parameter array. But, if we overload it, we can actually link the parameters together! See the below code for an example. 

In [ ]:
from pygama.math.functions.sum_dists import sum_dists


class cauchy_on_gauss_gen(sum_dists):
    def __init__(self):
        # we first create an array contains the indices of the parameter array
        # that will eventually be input to function calls
        (frac1, mu, sigma, mu2, sigma2) = range(5)

        # we now create an array containing the distributions and their shape parameters
        # The last item in a parameter array is the fraction or area component
        # The last item in the last parameter array corresponds to the total area if present
        args = [gaussian, [mu, sigma, frac1], cauchy, [mu2, sigma2, frac1, frac1]]
        # we initialize with the frac_flag = "fracs" to let the constructor know we are sending frac parameters only
        super().__init__(*args, frac_flag="fracs")

    def _link_pars(
        self,
        shape_par_idx,
        area_idx,
        frac_idx,
        total_area_idx,
        params,
        areas,
        fracs,
        total_area,
    ):
        shape_pars, cum_len, areas, fracs, total_area = super()._link_pars(
            shape_par_idx,
            area_idx,
            frac_idx,
            total_area_idx,
            params,
            areas,
            fracs,
            total_area,
        )

        fracs[1] = (
            1 - fracs[0]
        )  # create :math:`(1-frac1)` for the cauchy, and :math:`frac1` for the exgauss
        total_area[
            0
        ] = 1  # just overwrite the total area because we don't want to fit it

        return shape_pars, cum_len, areas, fracs, total_area

    def get_req_args(self) -> tuple[str, str, str]:
        r"""
        Return the required arguments for this instance
        """
        return "frac1", "mu", "sigma", "mu2", "sigma2"


cauchy_on_gauss = cauchy_on_gauss_gen()

### Again, that was painless; let's see how well it does fitting data

In [ ]:
c = cost.UnbinnedNLL(xmix, cauchy_on_gauss.norm_pdf)

m = Minuit(c, (xr[0], xr[1], 0.5, 0.1, 0.2, 1, 0.1))
m.fixed[0, 1] = True
m.limits[2] = (0, 1)
m.limits[3, 4, 5, 6] = (0.5, 2)
print(m.migrad())

plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(
    xm,
    cauchy_on_gauss.norm_pdf(xm, np.array(m.values)) * len(xmix) * dx[0],
    label="fit",
)
plt.plot(
    xm,
    cauchy_on_gauss.get_pdf(xm, np.array([0.5, mu, sigma, mu2, sigma2]))
    * len(xmix)
    * dx[0],
    label="actual",
)
plt.legend();

### The `both` flag in the `sum_dists` constructor 

What if we wanted to find the signal and background counts for distributions that share unequal fractions? I.e. what if we want to construct the following: 

$pdf = Af_1\cdot gauss\_{pdf}+B(1-f_1)\cdot cauchy\_{pdf}$

It's easy! We use the `both` flag! 

The `both` keywords causes `sum_dists` to look for ordering of parameters in each index array like `[shape, mu, sigma, area, frac]`, except the last parameter array index must have the form `[shape, mu, sigma, area, frac, total_area]`

In [ ]:
from pygama.math.functions.sum_dists import sum_dists


class cauchy_on_gauss_gen(sum_dists):
    def __init__(self):
        # we first create an array contains the indices of the parameter array
        # that will eventually be input to function calls
        (area1, frac1, mu, sigma, area2, mu2, sigma2) = range(7)

        # we now create an array containing the distributions and their shape parameters
        # The last item in a parameter array is the fraction or area component
        # The last item in the last parameter array corresponds to the total area if present
        args = [
            gaussian,
            [mu, sigma, area1, frac1],
            cauchy,
            [mu2, sigma2, area2, frac1, frac1],
        ]
        # we initialize with the frac_flag = "fracs" to let the constructor know we are sending frac parameters only
        super().__init__(*args, frac_flag="both")

    def _link_pars(
        self,
        shape_par_idx,
        area_idx,
        frac_idx,
        total_area_idx,
        params,
        areas,
        fracs,
        total_area,
    ):
        shape_pars, cum_len, areas, fracs, total_area = super()._link_pars(
            shape_par_idx,
            area_idx,
            frac_idx,
            total_area_idx,
            params,
            areas,
            fracs,
            total_area,
        )

        fracs[1] = (
            1 - fracs[0]
        )  # create :math:`(1-frac1)` for the gaussian, and :math:`htail` for the exgauss
        total_area[
            0
        ] = 1  # just overwrite the total area because we don't want to fit it

        return shape_pars, cum_len, areas, fracs, total_area

    def get_req_args(self) -> tuple[str, str, str]:
        r"""
        Return the required arguments for this instance
        """
        return "area1", "frac1", "mu", "sigma", "area2", "mu2", "sigma2"


cauchy_on_gauss = cauchy_on_gauss_gen()

In [ ]:
c = cost.ExtendedUnbinnedNLL(xmix, cauchy_on_gauss.pdf_ext)

m = Minuit(c, (xr[0], xr[1], 100, 0.1, 0.1, 0.3, 100, 1, 2))
m.fixed[0, 1] = True  # fix the x_lo, x_hi
m.limits[2, 6] = (0, None)
m.limits[3] = (0, 1)
m.limits[4, 5, 7, 8] = (0, 2)
print(m.migrad())

plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(xm, cauchy_on_gauss.pdf_ext(xm, np.array(m.values))[1] * dx[0], label="fit")
plt.legend();

#### Let's see if that last fit agrees with a fit from a function we would define "by hand" 

In [ ]:
def cauchy_gauss_pdf_ext(x, x_lo, x_hi, A, B, frac, mu, sigma, mu2, sigma2):
    return A * frac * np.diff(
        gaussian.get_cdf(np.array([x_lo, x_hi]), mu, sigma)
    ) + B * (1 - frac) * np.diff(
        cauchy.get_cdf(np.array([x_lo, x_hi]), mu2, sigma2)
    ), A * frac * gaussian.get_pdf(
        x, mu, sigma
    ) + (
        1 - frac
    ) * B * cauchy.get_pdf(
        x, mu2, sigma2
    )

In [ ]:
c = cost.ExtendedUnbinnedNLL(xmix, cauchy_gauss_pdf_ext)

m = Minuit(c, xr[0], xr[1], 100, 100, 0.1, 0.1, 0.3, 1, 2)
m.fixed[0, 1] = True  # fix the x_lo, x_hi
m.limits[2, 3] = (0, None)
m.limits[4] = (0, 1)
m.limits[5, 6, 7, 8] = (0, 2)
print(m.migrad())

plt.errorbar(cx, n, n**0.5, fmt="ok")
xm = np.linspace(*xr)
plt.plot(xm, cauchy_gauss_pdf_ext(xm, *m.values)[1] * dx[0], label="fit")
plt.legend();

We see that `sum_dists` reproduces results quickly and without the need for spending time writing new methods for each of the summed distributions. 

# Conclusion 
Hopefully you have learned how to use the distributions and tools packaged in `pygama` for your own scientific purposes. If there's a distribution you see is missing, feel free to contribute it using the conventions described above! 